In [99]:
from random import randint
import time
from time import sleep
import datetime
import yfinance as yf
import openpyxl
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
%matplotlib inline
from mystockrec import xlsx_df, export_xlsx, data_load_user_input, get_input

In [67]:
def data_load_user_input()
    df = xlsx_df('df__734_2021.10.07 00_06_06.xlsx')
    display(df.loc[0:3,'Unnamed: 0'])
    df.info()
    df.head()

    # Clean data set
    df['shortName'] = df['shortName'].astype('str')
    df['shortName'] = df['shortName'].apply(lambda x: x.upper().translate({ord(c): "" for c in "!?/[]|{}.:()'\""}))
    df.drop(['Unnamed: 0','Unnamed: 0.1'], axis=1, inplace=True)

    # Select columns
    df = df[['shortName','symbol','cluster','sector','Rank_CFoEV_RoA',
             'revenueGrowth','earningsGrowth','SalesoEV_in_%','beta']]
    #'EV_calc','EV_52high','EV_52low','SalesoEV_in_%','EBITDAoEV_in_%','CFoEV_in_%','Rank_SalesoEV','Rank_EBITDAoEV','Rank_CFoEV','Rank_RoA','Rank_CFoEV_RoA'
    #df = df[['sector', 'cluster','fullTimeEmployees', 'city', 'state', 'country', 'industry','EV_calc','EV_52high','EV_52low','SalesoEV_in_%','EBITDAoEV_in_%','CFoEV_in_%','Rank_SalesoEV','Rank_EBITDAoEV','Rank_CFoEV','Rank_RoA','Rank_CFoEV_RoA', 'ebitdaMargins', 'profitMargins', 'grossMargins', 'operatingCashflow', 'revenueGrowth', 'operatingMargins', 'ebitda', 'targetLowPrice', 'recommendationKey', 'grossProfits', 'freeCashflow', 'targetMedianPrice', 'currentPrice', 'earningsGrowth', 'returnOnAssets', 'numberOfAnalystOpinions', 'targetMeanPrice', 'debtToEquity', 'targetHighPrice', 'totalCash', 'totalDebt', 'totalRevenue', 'financialCurrency', 'recommendationMean', 'exchange', 'shortName', 'longName', 'symbol', 'market', 'enterpriseToRevenue', 'enterpriseToEbitda', 'sharesOutstanding', 'sharesShort', 'fundFamily', 'heldPercentInstitutions', 'priceToBook', 'heldPercentInsiders', 'beta', 'enterpriseValue', 'earningsQuarterlyGrowth', 'forwardPE', 'sharesShortPriorMonth', 'trailingAnnualDividendYield', 'payoutRatio', 'trailingAnnualDividendRate', 'dividendRate', 'currency', 'trailingPE', 'marketCap', 'fiftyTwoWeekHigh', 'fiftyTwoWeekLow']]

0    0
1    1
2    2
3    3
Name: Unnamed: 0, dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 734 entries, 0 to 733
Columns: 188 entries, Unnamed: 0 to cluster
dtypes: bool(2), float64(140), int64(10), object(36)
memory usage: 1.0+ MB


In [109]:
# Getting the user input
def get_input():
    br = '\n' # if needed variable for string literal line break, put {nl} if new line is needed
    shortName_input = input(" What company do you want to check? ")
    shortName_input = shortName_input.upper()
    comp_match = df[df['shortName'].str.contains(shortName_input)]
    if len(comp_match) == 1:
        print()
        print(f"{comp_match.loc[1:,'shortName'].values[0]} was found in the database.")
        time.sleep(1)
        print()
        print("Let's take a closer look.")
        
        time.sleep(1)
        print()
        print(f"{comp_match.loc[1:,'shortName'].values[0]} is in cluster" 
              f" {comp_match.loc[1:,'cluster'].values[0]:.0f}"
              #f"sector {comp_match.loc[1:,'sector'].values[0]} "
              f" and was ranked on overall position {comp_match.loc[1:,'Rank_CFoEV_RoA'].values[0]:.0f}"
              f" out of {len(df)} companies.")
        
        time.sleep(1)
        display(comp_match)
        
        time.sleep(1)
        print()
        print(f"Top ranked companies in cluster {comp_match.loc[1:,'cluster'].values[0]:.0f} are:")
        #display(df[df['cluster'].isin([int(comp_match.loc[1:,'cluster'])])].nsmallest(10,['Rank_CFoEV_RoA']))
        
        df_usr = df[df['cluster'].isin([int(comp_match.loc[1:,'cluster'])])].nsmallest(10,['Rank_CFoEV_RoA'])
        df_usr = df_usr.append(comp_match)
        df_usr.drop_duplicates(inplace=True)
        df_usr = df_usr.nsmallest(11,['Rank_CFoEV_RoA'])
        display(df_usr)
    elif len(comp_match) > 1:
        print()
        print(f"Several companies were found.")
        
        time.sleep(1)
        display(comp_match)
        symbol_input = input(f"Please select only one by typing the symbol of the company.")
        symbol_input = symbol_input.upper()
        try:
            comp_match = df.loc[df['symbol'] == symbol_input]
            print()
            print(f"{comp_match.loc[1:,'shortName'].values[0]} was found in the database.")
            
            time.sleep(1)
            print(f"{comp_match.loc[1:,'shortName'].values[0]} is in cluster"
                  f"{comp_match.loc[1:,'cluster'].values[0]:.0f}"
                  #f"sector {comp_match.loc[1:,'sector'].values[0]} "
                  f" and was ranked on overall position {comp_match.loc[1:,'Rank_CFoEV_RoA'].values[0]:.0f}"
                  f" out of {len(df)} companies.")
            time.sleep(1)
            display(comp_match)
            
            time.sleep(1)
            print()
            print(f"Top ranked companies in cluster {comp_match.loc[1:,'cluster'].values[0]} are:")
            #display(df[df['cluster'].isin([int(comp_match.loc[1:,'cluster'])])].nsmallest(10,['Rank_CFoEV_RoA']))
            
            df_usr = df[df['cluster'].isin([int(comp_match.loc[1:,'cluster'])])].nsmallest(10,['Rank_CFoEV_RoA'])
            df_usr = df_usr.append(comp_match)
            df_usr.drop_duplicates(inplace=True)
            df_usr = df_usr.nsmallest(11,['Rank_CFoEV_RoA'])
            display(df_usr)
        except:
            pass
    else:
        print('Sorry, your company is not in the database and has to be downloaded first, try again.')
        user_try_again = input('Do you want to try again? Type y for yes or n for no')
        if user_try_again == 'y':
            get_input()
        else:
            print("Thanks for using stock recommender")

In [110]:
get_input()

 What company do you want to check? apple

APPLE INC was found in the database.

Let's take a closer look.

APPLE INC is in cluster 5 and was ranked on overall position 61 out of 734 companies.


,shortName,symbol,cluster,sector,Rank_CFoEV_RoA,revenueGrowth,earningsGrowth,SalesoEV_in_%,beta
342,APPLE INC,AAPL,5.0,Technology,61.0,0.364,1.0,14.63664,1.222222



Top ranked companies in cluster 5 are:


,shortName,symbol,cluster,sector,Rank_CFoEV_RoA,revenueGrowth,earningsGrowth,SalesoEV_in_%,beta
212,QUIDEL CORPORATION,QDEL,5.0,Healthcare,2.0,-0.125,-0.710,34.887055,-0.073689
176,ADAIRS FPO,ADH.AX,5.0,Consumer Cyclical,3.0,0.231,-0.119,68.458984,1.376852
497,"HOLOGIC, INC",HOLX,5.0,Healthcare,4.0,0.420,0.962,27.790450,1.171397
669,"T ROWE PRICE GROUP, INC",TROW,5.0,Financial Services,5.0,0.363,0.357,17.524824,1.189956
285,"EMERGENT BIOSOLUTIONS, INC",EBS,5.0,Healthcare,6.0,0.782,NaN,51.643262,0.979869
634,QUEST DIAGNOSTICS INCORPORATED,DGX,5.0,Healthcare,9.0,0.396,2.647,51.326988,1.033842
206,LOGITECH INTERNATIONAL SA - R,LOGI,5.0,Technology,12.0,0.657,1.595,42.927705,0.633979
481,"GILEAD SCIENCES, INC",GILD,5.0,Healthcare,16.0,0.209,NaN,24.633191,0.413209
360,"H&R BLOCK, INC",HRB,5.0,Consumer Cyclical,17.5,0.287,1.162,60.370459,0.863982
660,SNAP-ON INCORPORATED,SNA,5.0,Industrials,17.5,0.444,1.032,37.157660,1.277292
